In [8]:
from time import time
from collections import defaultdict
from datetime import datetime
import multiprocessing
from pathlib import Path
import logging
import unicodedata

from botok import BoString
from botok.vars import CharMarkers
from openpecha.corpus.download import download_corpus
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

import config

logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

## Dataset

In [70]:
corpus = [
    {"path": config.DATA_DIR / "esukhia_katen", "pattern": "-tokenized"},  
    {"path": config.DATA_DIR / "literary_bo", "pattern": ".txt"}
]

In [72]:
def get_files(corpus):
    for corpus_item in corpus:
        for pecha_path in corpus_item["path"].iterdir():
            if not pecha_path.is_dir(): continue
            for fn in pecha_path.iterdir():
                if corpus_item["pattern"] not in fn.name: continue
                yield fn
            
def is_punt(string):
    normal_punt = CharMarkers(5)
    special_punt = CharMarkers(6)
    bo_string = BoString(string)
    for _, char_marker_value in bo_string.base_structure.items():
        char_maker = CharMarkers(char_marker_value)
        if char_maker == normal_punt or char_maker == special_punt:
            return True
    return False

def preprocess(token):
    token = token.replace("_", "")
    if not token.endswith("་"):
        token += "་"
    return token
    
def tokenize(text):
    return [preprocess(token) for token in text.split() if token and not is_punt(token)]
    
def get_sentences(files):
    for fn in files:
        for sentence in fn.open('r'):
            if len(sentence.split()) < 3: continue
            yield tokenize(unicodedata.normalize("NFKC", sentence.strip()))

### EDA (Exploratory Data Analysis)

In [74]:
files = list(get_files(corpus))
files[-5:]

[PosixPath('/home/studio-lab-user/.models/data/literary_bo/P8B4D67B5/UT3JT13350_018_0000.txt'),
 PosixPath('/home/studio-lab-user/.models/data/literary_bo/P8B4D67B5/UT3JT13350_019_0000.txt'),
 PosixPath('/home/studio-lab-user/.models/data/literary_bo/P8B4D67B5/UT3JT13350_020_0000.txt'),
 PosixPath('/home/studio-lab-user/.models/data/literary_bo/P8B4D67B5/UT3JT13350_021_0000.txt'),
 PosixPath('/home/studio-lab-user/.models/data/literary_bo/P9E02F6EB/UT3JT13313_001_0000.txt')]

In [76]:
import random
i = random.randint(0, len(files))

In [77]:
print(files[i].read_text()[:500]), files[i]
tokenize(files[i].read_text()[:500])

༼_༽ ན་མོ་ རཏྣ་ ཏྲ་ ཡ་ ཡ་ །_ རྒྱལ་སྲིད་ སྤངས་ ན་ དཀའ་བ་ བརྒྱ་ ཕྲག་ གིས་ །_ བླ་མ་ བརྒྱ་ དང་ ལྔ་ བཅུ་ རྩ་ བདུན་ བརྟེན་ །_
ཤེས་བྱ་ ཐམས་ཅད་ ཡང་དག་ ཐུགས་ སུ་ ཆུད་ །_
རྒྱལ་བ་ ཉིས་་ ར་ གྱུར་ ལ་ གསོལ་བ་ འདེབས་ །_
རྒྱལ་བ་ འི་ མདུན་ ན་ རྒྱལ་སྲས་ བཟང་པོ་ སྐྱོང་ །_
ཁ་བ་ ཅན་ དུ་ དཔལ་ལྡན་ སྨ་ར་ མེ་ མཇད་ །_ དགའ་ལྡན་ གནས་ སུ་ ནམ་མཁའ་ ཏྲི་ མ་ མེད་ །_
ཡིད་བཞིན་ ནོར་བུ་ ཁྱེད་ ལ་ གསོལ་བ་ འདེབས་ །_
སྒྲོལ་མ་ ས་ ལུང་བསྟན་ རྒྱལ་བུ་ དཀོན་མཆོག་ འབང་ །_ ཐུབ་བསྟན་ ཡོངས་སུ་ རྫོགས་པ་ འི་ མངའ་བདག་ མཆོག་ ལྷ་ཆོས་ བདུན་ ཇོ་བོ་ བཀ


['ན་མོ་',
 'རཏྣ་',
 'ཏྲ་',
 'ཡ་',
 'ཡ་',
 'རྒྱལ་སྲིད་',
 'སྤངས་',
 'ན་',
 'དཀའ་བ་',
 'བརྒྱ་',
 'ཕྲག་',
 'གིས་',
 'བླ་མ་',
 'བརྒྱ་',
 'དང་',
 'ལྔ་',
 'བཅུ་',
 'རྩ་',
 'བདུན་',
 'བརྟེན་',
 'ཤེས་བྱ་',
 'ཐམས་ཅད་',
 'ཡང་དག་',
 'ཐུགས་',
 'སུ་',
 'ཆུད་',
 'རྒྱལ་བ་',
 'ཉིས་་',
 'ར་',
 'གྱུར་',
 'ལ་',
 'གསོལ་བ་',
 'འདེབས་',
 'རྒྱལ་བ་',
 'འི་',
 'མདུན་',
 'ན་',
 'རྒྱལ་སྲས་',
 'བཟང་པོ་',
 'སྐྱོང་',
 'ཁ་བ་',
 'ཅན་',
 'དུ་',
 'དཔལ་ལྡན་',
 'སྨ་ར་',
 'མེ་',
 'མཇད་',
 'དགའ་ལྡན་',
 'གནས་',
 'སུ་',
 'ནམ་མཁའ་',
 'ཏྲི་',
 'མ་',
 'མེད་',
 'ཡིད་བཞིན་',
 'ནོར་བུ་',
 'ཁྱེད་',
 'ལ་',
 'གསོལ་བ་',
 'འདེབས་',
 'སྒྲོལ་མ་',
 'ས་',
 'ལུང་བསྟན་',
 'རྒྱལ་བུ་',
 'དཀོན་མཆོག་',
 'འབང་',
 'ཐུབ་བསྟན་',
 'ཡོངས་སུ་',
 'རྫོགས་པ་',
 'འི་',
 'མངའ་བདག་',
 'མཆོག་',
 'ལྷ་ཆོས་',
 'བདུན་',
 'ཇོ་བོ་',
 'བཀ་']

In [ ]:
sentences = list(get_sentences(files))

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/botok/textunits/bostring.py:82: UserWarning: Beware of unexpected results: input string contains the non-expanded char "ༀ", found in "".
  warn(


In [ ]:
len(sentences)

In [ ]:
def top_k_highest_freq_word(sentences, k=10):
    word_freq = defaultdict(int)
    for sent in sentences:
        for i in sent:
            word_freq[i] += 1
    return sorted(word_freq, key=word_freq.get, reverse=True)[:10]

top_k_highest_freq_word(sentences)

## Train model

In [ ]:
def train(sentences, model_name, epochs=30, checkpoint=None):
    
    # create the model
    model = Word2Vec(
        vector_size=100,
        window=5,
        min_count=1,
        min_alpha=0.0007, 
        negative=20,
        workers=multiprocessing.cpu_count()-1
    )
    
    # build Vocab
    t = time()
    model.build_vocab(sentences, progress_per=10000)
    print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))
    
    # start training
    t = time()
    model.train(sentences, total_examples=model.corpus_count, epochs=epochs, report_delay=1)
    print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))
    
    # surfix model with timestemp
    model_name = f"{model_name}-{datetime.now()}"
 
    # save the model
    models_path = config.MODELS_DIR / "word2vec"
    models_path.mkdir(exist_ok=True, parents=True)
    model_path = models_path / f"{model_name}.model"
    
    model.save(str(model_path))
    print(f"[INFO] Model save at {model_path}")
    
    # save only wordvectors
    word_vectors_path = models_path / f"{model_name}.wordvectors"
    
    # Store just the words + their trained embeddings.
    word_vectors = model.wv
    word_vectors.save(str(word_vectors_path))
    
    return word_vectors_path

In [ ]:
word_vectors_path = train(sentences, "literary_bo")

## Exploring the model

In [ ]:
# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load(str(word_vectors_path), mmap='r')

In [ ]:
wv.most_similar("སྟོབས་")

In [ ]:
wv.most_similar("མཛད་པ་")

In [ ]:
wv.most_similar("བླ་མ་")

In [ ]:
wv.most_similar("རྩ་བ་")

In [ ]:
wv.most_similar("ཉིད་")